##### Social Media Analytics
### Introduction to Text Mining
## Sentiment Analysis
(c) Nuno Antonio 2019-2022 v1.02

### Initial setup

In [1]:
# Import packages
import csv
import pandas as pd
import numpy as np
import nltk 
import re
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

In [2]:
# Load dataset
dtypes = {'RevID':'category','Source':'category','HotelID':'category',
  'HotelType':'category','HotelStars':'category','ObsDateGlobalRating':'float64',
  'Language':'category','RevUserName':'category','RevUserLocation':'category','RevOverallRating':'float64'}
ds = pd.DataFrame(pd.read_csv("HotelOnlineReviews.txt",sep="|", 
  error_bad_lines=False, dtype=dtypes, decimal=','))

C:\Users\User\AppData\Local\Temp\ipykernel_3668\473334055.py:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ds = pd.DataFrame(pd.read_csv("HotelOnlineReviews.txt",sep="|",
Skipping line 12799: expected 21 fields, saw 23

Skipping line 37247: expected 21 fields, saw 22



In [3]:
# Drop non-English reviews
ds = ds.drop(ds[ds.Language!='English'].index)

In [4]:
ds.head()

,RevID,Source,HotelID,HotelType,HotelStars,HotelRooms,ObsDate,ObsDateTotalReviewsOnSite,ObsDateGlobalRating,RevDescription,...,RevUsername,RevUserLocation,RevPublishedDate,RevOverallRating,RevLocationRating,RevSleepQualityRating,RevRoomsRating,RevServiceRating,RevValueRating,RevCleanlinessRating
0,B377,Booking,1,Resort,4,158,2015-11-29,1092,8.8,Hotel is so centrally located with bars and...,...,John,Ireland,2015-11-12,10.0,-1,-1,-1,-1,-1,-1
1,B378,Booking,1,Resort,4,158,2015-11-29,1092,8.8,Room were big & comfortable.þ Very noisy at ni...,...,David,United Kingdom,2015-11-09,5.8,-1,-1,-1,-1,-1,-1
2,B379,Booking,1,Resort,4,158,2015-11-29,1092,8.8,right near the beach & breakfast good!þ Noisy ...,...,Dawn,Portugal,2015-11-07,7.5,-1,-1,-1,-1,-1,-1
3,B380,Booking,1,Resort,4,158,2015-11-29,1092,8.8,"The location is the best in Portimao, just acr...",...,Ron,USA,2015-10-29,9.2,-1,-1,-1,-1,-1,-1
4,B381,Booking,1,Resort,4,158,2015-11-29,1092,8.8,Hotel was clean and the staff helpful and frie...,...,Pat,Spain,2015-10-27,7.1,-1,-1,-1,-1,-1,-1


### Functions

In [5]:
# Text preprocessing
def textPreProcess(rawText, removeHTML=True, charsToRemove = r'\?|\.|\!|\;|\.|\"|\,|\(|\)|\&|\:|\-', removeNumbers=True, removeLineBreaks=False, specialCharsToRemove = r'[^\x00-\xfd]', convertToLower=True, removeConsecutiveSpaces=True):
    if type(rawText) != str:
        return rawText
    procText = rawText
        
    # Remove HTML
    if removeHTML:
        procText = BeautifulSoup(procText,'html.parser').get_text()

    # Remove punctuation and other special characters
    if len(charsToRemove)>0:
        procText = re.sub(charsToRemove,' ',procText)

    # Remove numbers
    if removeNumbers:
        procText = re.sub(r'\d+',' ',procText)

    # Remove line breaks
    if removeLineBreaks:
        procText = procText.replace('\n',' ').replace('\r', '')

    # Remove special characters
    if len(specialCharsToRemove)>0:
        procText = re.sub(specialCharsToRemove,' ',procText)

    # Normalize to lower case
    if convertToLower:
        procText = procText.lower() 

    # Replace multiple consecutive spaces with just one space
    if removeConsecutiveSpaces:
        procText = re.sub(' +', ' ', procText)

    return procText

In [6]:
# Tokenize words
def tokenize_words(words):
    if (type(words) != str) or (word_tokenize(words) == ''):
        return np.nan
    else:
        return word_tokenize(words)

In [7]:
# Function to create text from words
def recreateText(words):
    if type(words) == list:
        temp_str = (' ').join(words)
        return temp_str
    else:
        return np.nan

In [8]:
# Function to break texts into sentences
def tokenize_sentences(texts):
    s_token = sent_tokenize(texts)
    return s_token

In [9]:
# Function to remove stop words
def removeStopWords(t, stop_words):
    if type(t) == list:
        return [w for w in t if not w in stop_words]
    else:
        return np.nan

### Analysis

In [11]:
# Because a review can express multiple opinions, let's analyze opinions by sentence

# Break reviews' into a list of lists sentences
listOfSentences = ds.RevDescription.apply(tokenize_sentences)

In [10]:
# Create a dataframe with only the description
processedReviews = pd.DataFrame(data=ds.RevDescription.apply(textPreProcess,charsToRemove ='', removeLineBreaks=False, removeNumbers=False).values, index=ds.index, columns=['PreProcessedText'])

/Users/nunoantonio/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [11]:
# Check first review
ds.RevDescription[0]

'Hotel is  so  centrally located with  bars and restaurants all  within a few minutes walk. Taxis,beach etc. literally on the doorstep. Breakfast excellent. Staff very  friendly and helpful. Will definitely be going  back.þ No  socket  at worktop for electric  kettle so had to place  it on the floor to boil Electric hob not  in use. Shortage of basic  kitchen items to make snack but  we  managed ok.'

In [12]:
# Sentences of first review
listOfSentences[0]

['Hotel is  so  centrally located with  bars and restaurants all  within a few minutes walk.',
 'Taxis,beach etc.',
 'literally on the doorstep.',
 'Breakfast excellent.',
 'Staff very  friendly and helpful.',
 'Will definitely be going  back.þ No  socket  at worktop for electric  kettle so had to place  it on the floor to boil Electric hob not  in use.',
 'Shortage of basic  kitchen items to make snack but  we  managed ok.']

In [13]:
# Create DataFrame for sentences
sentences = pd.DataFrame(data=[item for elem in listOfSentences for item in elem], columns=['BaseText'])

In [14]:
# Add a column with the review ID
sentencesPerReview = []
for elem in listOfSentences:
  sentencesPerReview.append(len(elem))
sentences['RevID'] = np.repeat(ds['RevID'].values,sentencesPerReview)

In [15]:
# Preprocess text 
sentences['PreProcessedText'] = sentences['BaseText'].apply(textPreProcess)

/Users/nunoantonio/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [16]:
# Get words
sentences['Words'] =  sentences['PreProcessedText'].apply(tokenize_words)

In [17]:
# Remove stopwords
stop_words = set(stopwords.words('english'))
sentences['WordsCleaned'] = sentences['Words'].apply(removeStopWords,stop_words=stop_words)

In [18]:
# Recreate sentence without stopwords
sentences['ProcessedText'] = sentences['WordsCleaned'].apply(recreateText)

In [19]:
# Create sentiment analysis object
analyser = SentimentIntensityAnalyzer()

In [20]:
# To test, let's evaluate first sentence of first review
# Scales:
#   compound: -1:most extreme negative, 1:most extreme positive
#     positive: compound >=0.05
#     neutral: -0.05<compound<0.05
#     negative: compound <= -0.05
#   pos, neu, neg: proportion of text that are positive, neutral or negative
score = analyser.polarity_scores(sentences['ProcessedText'][0])
print(sentences['ProcessedText'][0],score)

hotel centrally located bars restaurants within minutes walk {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [21]:
# Process sentiment for all sentences
all_scores = []
for t in (sentences['ProcessedText'][:]):
  score = analyser.polarity_scores(t)
  all_scores.append(score)
sentences['Sentiment'] = [c['compound'] for c in all_scores]

In [22]:
# Compute review's sentiment as the mean sentiment from its sentences
meanByReview = sentences.groupby('RevID')['Sentiment'].mean()

# Consider sentences with no result as neutral (0)
meanByReview = meanByReview.fillna(0)

# Add column Sentiment to reviews Dataframe
ds['Sentiment'] = meanByReview[ds['RevID']].values

In [23]:
# Assign a qualitative evaluation to the review
bins = pd.IntervalIndex.from_tuples([(-1.1, -0.05), (-0.05, 0.05), (0.05, 1)], closed='right')
x = pd.cut(ds['Sentiment'].to_list(), bins)
x.categories = ['Negative','Neutral','Positive']
ds['Polarity'] = x

In [24]:
# Analysis examples:
# Mean by hotel 
ex1 = ds.groupby('HotelID')['Sentiment'].mean().to_frame()
ex1

,Sentiment
HotelID,
1,0.361068
10,0.376767
11,0.323291
12,0.339835
13,0.369670
...,...
65,0.478573
66,0.452675
7,0.240772


In [25]:
# Analysis examples:
# Mean by hotel stars and type
ex2 = ds[['HotelType','HotelStars','Sentiment']].groupby(['HotelType','HotelStars'], as_index=False).mean()
ex2

,HotelType,HotelStars,Sentiment
0,City,2,0.236023
1,City,3,0.334388
2,City,4,0.343244
3,City,5,0.324848
4,Resort,2,0.302441
5,Resort,3,0.331119
6,Resort,4,0.406917
7,Resort,5,0.369157
